In [1]:
#webscraping here

import pandas as pd
import praw

reddit = praw.Reddit(client_id="69xO_MRhRBIQGprtVEZIbA",
    client_secret="GgPpkwrp_BPNduxYQ-dOcxfmrykfhQ",
    user_agent="test")


subreddit = reddit.subreddit("scu")

posts_dict = {"Body": [], "ID": [], "Ups": []}

for post in subreddit.hot(limit=1000):
    # Title and body of each post
    title = post.title
    text = post.selftext
    body = title + "\n" + text
    posts_dict['Body'].append(body)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Ups"].append(post.score)

top_posts = pd.DataFrame(posts_dict)
top_posts.to_csv("Top Posts.csv", index=True)
display(top_posts)

,Body,ID,Ups
0,Recruitments for Studies/Job Opportunities & O...,oouus2,17
1,Advice Paper for Incoming COEN/CS Students\nHi...,xkfy3u,43
2,GradDip Early Childhood Education \nHas anybod...,1ddurtk,1
3,Does the Honors Program just not tell the alte...,1ddl2x5,2
4,Robotics and Automation in SCU\nI am from Asia...,1dbiqsj,1
...,...,...,...
970,Freshman dorms\nHow hard is it to get into Fin...,kxpxqv,5
971,(Open to all Bay Area Residents) 1 Min Survey ...,kwq8y5,6
972,Hackathon Opportunity For SCU Students (All Ma...,ku4vj1,10
973,"Calling all engineering, computer science, and...",ks6uiv,4


In [ ]:

id_list = top_posts["ID"].tolist()
comments = pd.DataFrame()
comment_list = []
for i in id_list:                      #indicates start of the subission id processing
    submission = reddit.submission(i)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if(comment.author == "AutoModerator"):
            continue
        else:
            comment_list.append(comment.__dict__)

comments_df = pd.DataFrame(comment_list)

comments_df_cleaned = comments_df.filter(['_submission','id','author', 'body', 'ups'], axis=1)

comments_df_cleaned.to_csv(f'comments.csv')

display(comments_df_cleaned)

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize

nltk.download('punkt')


tokenizer = RegexpTokenizer(r'\w+')

dorm_names = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin-walsh', 'casa italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university villas', 'nobili', 'dunne']
dorm_columns = ['graham', 'finn', 'sanfilippo', 'campisi', 'mclaughlin_walsh', 'casa_italiana', 'sobrato', 'swig', 'flip', 'camp', 'casa', 'walsh', 'university_villas', 'nobili', 'dunne']

comments_df = pd.read_csv('comments.csv')
posts_df = pd.read_csv('Top Posts.csv')

# Function to extract dorm names from comments
def dorm_appears(comment, dorm):
    comment_lower = comment.lower()
    tokens = tokenizer.tokenize(comment_lower)
    return dorm in tokens

# Check if at least one dorm is found in the comment
def any_dorm_found(row):
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_") 
        if row[dorm_column]:
            return True
    return False

def data_cleaning(df, filename, body_col_name):
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
        df[dorm_column] = df[body_col_name].apply(dorm_appears, dorm=dorm)

    # Remove rows where no dorms  found
    df = df[df.apply(any_dorm_found, axis=1)]


    # Splitting comments with more than one dorm
    new_rows = []
    for col, row in df.iterrows():
        sentences = sent_tokenize(row[body_col_name])
        if sum(row[dorm_columns]) > 1:
            for sentence in sentences:
                new_row = row.copy()
                new_row[body_col_name] = sentence
                new_rows.append(new_row)
        else:
            new_rows.append(row)

    new_df = pd.DataFrame(new_rows)

    # After new rows added re check sentences for dorm names
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_")  # Ensure valid column names
        new_df[dorm_column] = new_df[body_col_name].apply(dorm_appears, dorm=dorm)

    # Remove rows where no dorms found
    new_df = new_df[new_df.apply(any_dorm_found, axis=1)]


    new_df.to_csv(f'{filename}')

data_cleaning(comments_df, 'comments_cleaned.csv', 'body')
data_cleaning(posts_df, 'posts_cleaned.csv', 'Body')


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess_and_tokenize(text):
    # Clean the text as previously defined
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    cleaned_text = " ".join(new_text)
    
    # Tokenize and ensure consistent input size with padding and truncation
    return tokenizer(cleaned_text, padding="max_length", max_length=128, truncation=True, return_tensors='pt')
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

def sent_analyze(text):
    try:
        encoded_input = preprocess_and_tokenize(text)
        output = model(**encoded_input)
        scores = output.logits.detach().numpy()
        scores = softmax(scores, axis=1)
        return scores[0]  # Return the first batch as you're processing texts one by one
    except Exception as e:
        print(f"Error processing text: {text}")
        print(e)
        return [0, 0, 0]  # Return a default value in case of error


In [4]:
#adding sentiment analysis to the both dfs

import pandas as pd
from scipy.special import softmax

# Apply the sentiment analysis function to each row in the 'Body' column
posts_cleaned = pd.read_csv('posts_cleaned.csv')
posts_cleaned["mcwalsh"] = posts_cleaned["mclaughlin_walsh"] | posts_cleaned["walsh"]
posts_cleaned["campisi_camp"] = posts_cleaned["campisi"] | posts_cleaned["camp"]
posts_cleaned["sanfilippo_flip"] = posts_cleaned["sanfilippo"] | posts_cleaned["flip"]
posts_cleaned.drop(columns=["walsh", "camp", "flip", "campisi", "sanfilippo", "mclaughlin_walsh"], inplace=True)



posts_cleaned['sentiment'] = posts_cleaned['Body'].apply(sent_analyze)

# Create new columns for negative, neutral, and positive scores by extracting the respective indices from the sentiment list
posts_cleaned['neg'] = posts_cleaned['sentiment'].apply(lambda x: x[0])
posts_cleaned['neu'] = posts_cleaned['sentiment'].apply(lambda x: x[1])
posts_cleaned['pos'] = posts_cleaned['sentiment'].apply(lambda x: x[2])

posts_cleaned.drop('sentiment', axis=1, inplace=True)

# View the DataFrame to confirm the new columns are added correctly
display(posts_cleaned)

#turn comments_cleaned.csv into a df then do the same as above for new df
comments_cleaned_df = pd.read_csv('comments_cleaned.csv')
comments_cleaned_df["mcwalsh"] = comments_cleaned_df["mclaughlin_walsh"] | comments_cleaned_df["walsh"]
comments_cleaned_df["campisi_camp"] = comments_cleaned_df["campisi"] | comments_cleaned_df["camp"]
comments_cleaned_df["sanfilippo_flip"] = comments_cleaned_df["sanfilippo"] | comments_cleaned_df["flip"]
comments_cleaned_df.drop(columns=["walsh", "camp", "flip", "campisi", "sanfilippo", "mclaughlin_walsh"], inplace=True)

comments_cleaned_df['sentiment'] = comments_cleaned_df['body'].apply(sent_analyze)
comments_cleaned_df['neg'] = comments_cleaned_df['sentiment'].apply(lambda x: x[0])
comments_cleaned_df['neu'] = comments_cleaned_df['sentiment'].apply(lambda x: x[1])
comments_cleaned_df['pos'] = comments_cleaned_df['sentiment'].apply(lambda x: x[2])
comments_cleaned_df.drop('sentiment', axis=1, inplace=True)
display(comments_cleaned_df)




,Unnamed: 0.1,Unnamed: 0,Body,ID,Ups,graham,finn,casa_italiana,sobrato,swig,casa,university_villas,nobili,dunne,mcwalsh,campisi_camp,sanfilippo_flip,neg,neu,pos
0,40,40,What are my chances?\nHey everyone!\n\nI'm cur...,1cp9tqh,4,False,False,False,False,False,False,False,False,False,False,True,False,0.010929,0.473358,0.515713
1,51,51,Graham/Finn\nDo incoming freshman have a chanc...,1cmhmzn,3,True,True,False,False,False,False,False,False,False,False,False,False,0.015283,0.955302,0.029415
2,69,69,"Transfer Housing Information\nHello, I'm a tra...",1chxhvo,3,False,True,False,False,False,False,False,False,False,False,False,False,0.018936,0.893320,0.087744
3,78,78,Orientation Sessions are open for registration...,1cd3a17,6,False,False,False,False,True,False,False,False,False,False,False,False,0.006647,0.892695,0.100658
4,82,82,Dunne bathroom updated?,1camxec,1,False,False,False,False,False,False,False,False,True,False,False,False,0.034072,0.938106,0.027822
5,82,82,Does anyone know if Dunne’s bathroom is somewh...,1camxec,1,False,False,False,False,False,False,False,False,True,True,False,False,0.012646,0.964606,0.022749
6,103,103,"Santa Clara University or UC Santa Cruz, Which...",1c22i9j,2,False,False,False,False,False,False,False,False,False,False,False,True,0.012864,0.678406,0.308730
7,116,116,Complaining about Swig & Swig Residents until ...,1byptue,2,False,False,False,False,True,False,False,False,False,False,False,False,0.382792,0.564780,0.052428
8,158,158,I got accepted to the SCU honors program so I ...,1bmxuwd,2,True,False,False,False,False,True,False,False,True,False,False,False,0.004542,0.680883,0.314575
9,158,158,"From what I’ve seen, Graham seems like by far ...",1bmxuwd,2,True,False,False,False,False,False,False,False,False,False,False,False,0.062990,0.446922,0.490088


,Unnamed: 0.1,Unnamed: 0,_submission,id,author,body,ups,graham,finn,casa_italiana,...,casa,university_villas,nobili,dunne,mcwalsh,campisi_camp,sanfilippo_flip,neg,neu,pos
0,54,54,1d5yi68,l6ostw4,Ch8ngus,Mo clue about being trans but graham and Finn ...,6,True,True,False,...,False,False,False,False,False,False,False,0.011656,0.081077,0.907267
1,55,55,1d5yi68,l6oy0fr,5torm,Swig is “sustainability” and Casa is “Italian”).,4,False,False,False,...,True,False,False,False,False,False,False,0.057658,0.790006,0.152336
2,57,57,1d5yi68,l6seh8v,Affectionate_Dog_600,"Swig, you will find it very inclusive there",-1,False,False,False,...,False,False,False,False,False,False,False,0.005366,0.099713,0.894921
3,132,132,1cqr2oq,l576uig,Skicrazy85,"Each dorm has its own personality, I was in So...",2,False,False,False,...,False,False,False,False,False,False,False,0.131188,0.789410,0.079402
4,170,170,1cm4xod,l36tyz2,Magellan_8888,It will be quite easy meeting people. I’m curr...,4,False,False,False,...,True,False,False,False,False,False,False,0.004078,0.137564,0.858359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,4021,4021,kxpxqv,gjbwypo,iEshkker,Graham is pretty hard to get into mostly becau...,3,True,False,False,...,False,False,False,False,False,False,False,0.046387,0.396804,0.556810
289,4023,4023,kxpxqv,gjenjk9,Bronco687,Swig fall 2021 bouta be crazy as fuck,1,False,False,False,...,False,False,False,False,False,False,False,0.117957,0.396551,0.485491
290,4024,4024,kxpxqv,gjdsg09,waffleseeds,I got into Dunne for when they thought COVID w...,1,False,False,False,...,False,False,False,True,False,False,False,0.250294,0.657244,0.092462
291,4024,4024,kxpxqv,gjdsg09,waffleseeds,Then when they were going to do one person per...,1,True,False,False,...,False,False,False,False,False,False,False,0.065666,0.900567,0.033767


In [6]:

#rankings based on sentiment analysis

#go through top posts and cleaned_comments ad avg the neg, neu, pos scores for each dorm
combined_dorm_names = dorm_names = ['graham', 'finn', 'sanfilippo_flip', 'campisi_camp', 'mcwalsh', 'casa italiana', 'sobrato', 'swig', 'casa', 'university villas', 'nobili', 'dunne']
dorm_sentiment = {}
for dorm in combined_dorm_names:
    dorm_column = dorm.replace(" ", "_").replace("-", "_")
    dorm_posts = posts_cleaned[posts_cleaned[dorm_column]]
    dorm_comments = comments_cleaned_df[comments_cleaned_df[dorm_column]]
    if(len(dorm_posts) + len(dorm_comments) == 0):
            continue
    else:
        dorm_sentiment[dorm] = {
            "neg": [(dorm_posts["neg"].mean() * len(dorm_posts)/(len(dorm_posts) + len(dorm_comments))) + (dorm_comments["neg"].mean() * len(dorm_comments)/(len(dorm_posts) + len(dorm_comments)))],
            "neu": [(dorm_posts["neu"].mean() * len(dorm_posts)/(len(dorm_posts) + len(dorm_comments))) + (dorm_comments["neu"].mean() * len(dorm_comments)/(len(dorm_posts) + len(dorm_comments)))],
            "pos": [(dorm_posts["pos"].mean() * len(dorm_posts)/(len(dorm_posts) + len(dorm_comments))) + (dorm_comments["pos"].mean() * len(dorm_comments)/(len(dorm_posts) + len(dorm_comments)))]
        }


display(dorm_sentiment)

#print ranking based on sentiment analysis: dorms with highest positive sentiment
dorm_sentiment_df = pd.DataFrame(dorm_sentiment)
dorm_sentiment_df = dorm_sentiment_df.transpose()

#df in positive order
dorm_sentiment_df = dorm_sentiment_df.sort_values(by='pos', ascending=False)
display(dorm_sentiment_df)
print("Ranking based on maximum positive sentiment:")
for i in range(len(dorm_sentiment_df)):
    print(f"{i+1}. {dorm_sentiment_df.index[i]}")

#df in negative order
dorm_sentiment_df = dorm_sentiment_df.sort_values(by='neg', ascending=True)
display(dorm_sentiment_df)
print("Ranking based on minimum negative sentiment:")
for i in range(len(dorm_sentiment_df)):
    print(f"{i+1}. {dorm_sentiment_df.index[i]}")

#df in neutral order
dorm_sentiment_df = dorm_sentiment_df.sort_values(by='neu', ascending=False)
display(dorm_sentiment_df)
print("Ranking based on maximum neutral sentiment:")
for i in range(len(dorm_sentiment_df)):
    print(f"{i+1}. {dorm_sentiment_df.index[i]}")


{'graham': {'neg': [0.0884923641099019],
  'neu': [0.5826141184635377],
  'pos': [0.328893491056528]},
 'finn': {'neg': [0.08701974734083398],
  'neu': [0.5942232655240344],
  'pos': [0.3187569857804806]},
 'sanfilippo_flip': {'neg': [0.2389320358633995],
  'neu': [0.5681010633707047],
  'pos': [0.1929668877273798]},
 'campisi_camp': {'neg': [0.20995082103070759],
  'neu': [0.4732359732900347],
  'pos': [0.31681322199957707]},
 'mcwalsh': {'neg': [0.09431889911110584],
  'neu': [0.7408798107734094],
  'pos': [0.16480126747718224]},
 'sobrato': {'neg': [0.09470943764254854],
  'neu': [0.6384985076414573],
  'pos': [0.2667920496818182]},
 'swig': {'neg': [0.25357519521200955],
  'neu': [0.506265555038925],
  'pos': [0.24015923090710126]},
 'casa': {'neg': [0.08500335130252336],
  'neu': [0.5837537520810178],
  'pos': [0.3312428530893828]},
 'nobili': {'neg': [0.10231983169381106],
  'neu': [0.6694359694208418],
  'pos': [0.22824419928448542]},
 'dunne': {'neg': [0.1354512392154223],
  'n

,neg,neu,pos
casa,[0.08500335130252336],[0.5837537520810178],[0.3312428530893828]
graham,[0.0884923641099019],[0.5826141184635377],[0.328893491056528]
finn,[0.08701974734083398],[0.5942232655240344],[0.3187569857804806]
campisi_camp,[0.20995082103070759],[0.4732359732900347],[0.31681322199957707]
dunne,[0.1354512392154223],[0.5747361839572086],[0.2898126387897926]
sobrato,[0.09470943764254854],[0.6384985076414573],[0.2667920496818182]
swig,[0.25357519521200955],[0.506265555038925],[0.24015923090710126]
nobili,[0.10231983169381106],[0.6694359694208418],[0.22824419928448542]
sanfilippo_flip,[0.2389320358633995],[0.5681010633707047],[0.1929668877273798]
mcwalsh,[0.09431889911110584],[0.7408798107734094],[0.16480126747718224]


Ranking based on maximum positive sentiment:
1. casa
2. graham
3. finn
4. campisi_camp
5. dunne
6. sobrato
7. swig
8. nobili
9. sanfilippo_flip
10. mcwalsh


,neg,neu,pos
casa,[0.08500335130252336],[0.5837537520810178],[0.3312428530893828]
finn,[0.08701974734083398],[0.5942232655240344],[0.3187569857804806]
graham,[0.0884923641099019],[0.5826141184635377],[0.328893491056528]
mcwalsh,[0.09431889911110584],[0.7408798107734094],[0.16480126747718224]
sobrato,[0.09470943764254854],[0.6384985076414573],[0.2667920496818182]
nobili,[0.10231983169381106],[0.6694359694208418],[0.22824419928448542]
dunne,[0.1354512392154223],[0.5747361839572086],[0.2898126387897926]
campisi_camp,[0.20995082103070759],[0.4732359732900347],[0.31681322199957707]
sanfilippo_flip,[0.2389320358633995],[0.5681010633707047],[0.1929668877273798]
swig,[0.25357519521200955],[0.506265555038925],[0.24015923090710126]


Ranking based on minimum negative sentiment:
1. casa
2. finn
3. graham
4. mcwalsh
5. sobrato
6. nobili
7. dunne
8. campisi_camp
9. sanfilippo_flip
10. swig


,neg,neu,pos
mcwalsh,[0.09431889911110584],[0.7408798107734094],[0.16480126747718224]
nobili,[0.10231983169381106],[0.6694359694208418],[0.22824419928448542]
sobrato,[0.09470943764254854],[0.6384985076414573],[0.2667920496818182]
finn,[0.08701974734083398],[0.5942232655240344],[0.3187569857804806]
casa,[0.08500335130252336],[0.5837537520810178],[0.3312428530893828]
graham,[0.0884923641099019],[0.5826141184635377],[0.328893491056528]
dunne,[0.1354512392154223],[0.5747361839572086],[0.2898126387897926]
sanfilippo_flip,[0.2389320358633995],[0.5681010633707047],[0.1929668877273798]
swig,[0.25357519521200955],[0.506265555038925],[0.24015923090710126]
campisi_camp,[0.20995082103070759],[0.4732359732900347],[0.31681322199957707]


Ranking based on maximum neutral sentiment:
1. mcwalsh
2. nobili
3. sobrato
4. finn
5. casa
6. graham
7. dunne
8. sanfilippo_flip
9. swig
10. campisi_camp


In [7]:

#search function
#output the body of comments and posts that contain the search term

def search(term):
    term = term.lower()
    term = term + " "
    search_results = []
    for i in range(len(posts_cleaned)):
        if term in posts_cleaned.iat[i,2].lower():
            search_results.append(posts_cleaned.iat[i,2])
    for i in range(len(comments_cleaned_df)):
        if term in comments_cleaned_df.iat[i,5].lower():
            search_results.append(comments_cleaned_df.iat[i,5] )
    display(search_results)

search("casa")


['If I was requesting casa could I choose to not select the small singles or would it just be entirely random?',
 'Swig is “sustainability” and Casa is “Italian”).',
 'It will be quite easy meeting people. I’m currently a junior, but the civil engineers at scu are perhaps the most social out of all the STEM majors in my grade. People in other classes are also social - I’m catching up on GE credits, so I’m taking classes with some freshman and sophomores. I think as long as you yourself are social it won’t be a problem. If you live on campus, there’s a decent shot the school will put you in a dorm with other transfer(s). I lived in casa last year (8 person apartment housing) and there was literally a suite of all transfer students.',
 'I don’t think freshmen can live in the casa singles but I might be wrong … I would choose graham',
 'If you get McWalsh or Dunne, it will be difficult to get Graham or Casa the next year because those who had it in freshman year will choose to stay again 

In [63]:
#apriori algorithm
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
comments_aa = comments_cleaned_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', '_submission', 'id', 'author', 'ups', 'neg', 'neu', 'pos'])
#display(comments_aa)
posts_aa = posts_cleaned.drop(['Unnamed: 0','Ups', 'Unnamed: 0.1', 'ID', 'neg', 'neu', 'pos'], axis = 1)
posts_aa = posts_aa.rename(columns={'Body': 'body'})
#display(posts_aa)
merged = pd.concat([posts_aa, comments_aa])

tags = ['good', 'awesome', 'bad', 'terrible', 'great', 'clean', 'dirty', 'nice', 'hate', 'hated', 'like', 'liked', 'love', 'loved', 'party']
# Function to extract dorm names from comments
def word_appears(comment, word):
    comment_lower = comment.lower()
    tokens = tokenizer.tokenize(comment_lower)
    tokens = [re.sub(r'^Ġ', '', token) for token in tokens]
    return word in tokens

# Check if at least one dorm is found in the comment
def any_word_found(row):
    for dorm in dorm_names:
        dorm_column = dorm.replace(" ", "_").replace("-", "_") 
        if row[dorm_column]:
            return True
    return False

for word in tags:
    merged[word] = merged['body'].apply(word_appears, word=word)

merged.to_excel("merged.xlsx")  

merged = merged.drop(columns=['body'])
merged = merged[~merged.all(axis=1)]
display(merged)

from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
itemsets = apriori(merged, min_support=0.01, use_colnames=True)

from mlxtend.frequent_patterns import association_rules
rules = association_rules(itemsets, metric="confidence", min_threshold=0.7)
rules = rules.filter(['antecedents', 'consequents', 'support', 'confidence', 'lift'])
print(rules)
rules.to_csv('rules.csv')




[nltk_data] Downloading package punkt to /Users/dshivani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,graham,finn,casa_italiana,sobrato,swig,casa,university_villas,nobili,dunne,mcwalsh,...,clean,dirty,nice,hate,hated,like,liked,love,loved,party
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
289,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
290,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
291,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


                antecedents              consequents   support  confidence  \
0                   (party)                   (swig)  0.028653    0.833333   
1           (finn, sobrato)                 (graham)  0.022923    0.888889   
2         (graham, sobrato)                   (finn)  0.022923    1.000000   
3              (finn, swig)                 (graham)  0.020057    1.000000   
4            (graham, swig)                   (finn)  0.020057    0.777778   
..                      ...                      ...       ...         ...   
65  (casa, nobili, sobrato)           (finn, graham)  0.011461    1.000000   
66  (casa, graham, sobrato)           (finn, nobili)  0.011461    1.000000   
67         (graham, nobili)    (finn, sobrato, casa)  0.011461    1.000000   
68           (casa, nobili)  (finn, graham, sobrato)  0.011461    1.000000   
69        (nobili, sobrato)     (finn, graham, casa)  0.011461    1.000000   

         lift  
0    2.403581  
1    3.485643  
2    4.532468  